In [15]:
# В блокноте производится предобработка исходных данных асмпп по одному мвн.

# Предобработка заключается в следующем:
# 1. Выделение в исходных данных отдельных рейсов.
# 2. Выделение всех возможных вариантов поостановочной трассы.
# 3. Разделение по временным периодам. Выделены следующие категории временных периодов:
#    - раб/вых -- разделение в соответсвии со справочником рабочих дней;
#    - лето/зима -- лето: с 1 мая по 30 сентября, зима с 1 октября по 30 апреля (можно менять);
#    - час суток ("ЧЧ:ММ-ЧЧ:ММ") -- привязка рейса к часу суток осуществляется по времени прибытия на первый о.п. трассы
#    Временной период задается строкой вида <сезон>-<тип дня>-<час суток>, например, "лето-раб-08:00-08:59"
# 3. Вычисление числа рейсов по каждому варианту поостановочной трассы в указанный временной период.
# 4. Вычисление посадки-высадки среднего рейса.
# 
# В результате создается набор таблиц, содержащих следующие сведения:
# 1. Таблица inout -- посадка-высадка:
#    - mvn -- название мвн;
#    - trace_id -- идентификационный номер варианта поостановочной трассы (см. табл. traces);
#    - period_type -- тип временного периодов;
#    - stop_sequence -- порядковый номер о.п.;
#    - stop_id -- идентификационный номер о.п.;
#    - stop_name -- наименование о.п.;
#    - pass_in -- посадка;
#    - pass_out -- высадка.
# 2. Таблица traces -- варианты поостановочных трасс:
#    - trace_id -- идентификационный номер варианта поостановочной трассы;
#    - n_races -- общее число рейсов в исходных данных асмпп;
#    - date_first -- дата начала первого рейса в исходных данных асмпп;
#    - date_last -- дата начала последнего рейса в исходных данных асмпп;
#    - flag -- флаг использования данных о посаждке-высадке для построения маршрутной матрицы (1 -- используется);
#    - trip_id -- перечень через запятую trip_id имеющих указанную поостановочную трассу;
#    - trace -- поостановочная трасса составленная из последовательного перечня "<stop_sequence>-<stop_id>", разделенного ",\n".
# 3. Таблица nraces -- число рейсов по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число рейсов в исходных данных асмпп.
# 4. Таблица pass -- число перевезенных пассажиров в среднем рейсе по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число пассажиров в среднем рейсе асмпп в формате "<среднее число пассажиров> ± <ср. кв. откл.>".
#


In [16]:
import os
import sys
sys._enablelegacywindowsfsencoding()
import numpy as np
import pandas as pd
%load_ext iimport

The iimport extension is already loaded. To reload it, use:
  %reload_ext iimport


In [17]:
args = {
    # директория с исходными данными асмпп
    'source' : 'C:/Users/poletaev_im/Correspondence-Lipetsk/result/1-Обратное(37)-вечер.csv',
    # справочник выходных и рабочих дней
    'workdays' : 'C:/Users/poletaev_im/ASM PP/workdays.csv',
    # задаем сезоны ММ.ДД-ММ.ДД
    'summer' : '05.01-09.30',
    'winter' : '10.01-04.30',
    # задаем список границ временных интервалов в формате ["ЧЧ:ММ", "ЧЧ:ММ",..., "ЧЧ:ММ"]
    'time' : ['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00',
              '12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    
    # директория с результатами работы
    'results' : 'C:/Users/poletaev_im/Correspondence-Lipetsk/result/обработаные/',
}

In [18]:
%def main(args):

In [19]:
# справочник рабочих дней
df_workdays = pd.read_csv(args['workdays'], sep=';', date_parser=True)
df_workdays['dt'] = pd.to_datetime(df_workdays.dt, format="%d.%m.%Y")
df_workdays['date'] = df_workdays.dt.dt.date
df_workdays.replace({'workday':{0:'вых',1:'раб'}}, inplace=True)

In [20]:
mvn = args['source'].split('/')[-1][:-4]

In [21]:
# загружаем данные асмпп по одному мвн
df_asmpp = pd.read_csv(args['source'], sep=';', encoding='cp1251')
# убираем записи имеющие некорректную дату
#df_asmpp = df_asmpp.query('fact_trip_date' > '20100101')

In [22]:
df_asmpp

,route_id,trip_id,survey_id,survey_start_date,survey_end_date,stop_id,arrival_time,pass_in,pass_out,vehicle_id,grafic,trip_num,stop_sequence,fact_trip_date,stop_name,mvn
0,310001,310001,310001,20180614,20180614,10231,17:01:23,13,0,310001,310001,37,1,20180614,пос. Северный Рудник,1-Обратное(37)-вечер
1,310001,310001,310001,20180614,20180614,10232,17:03:59,5,1,310001,310001,37,2,20180614,Северный Рудник,1-Обратное(37)-вечер
2,310001,310001,310001,20180614,20180614,10233,17:06:43,6,0,310001,310001,37,3,20180614,Птицефабрика Липецкая,1-Обратное(37)-вечер
3,310001,310001,310001,20180614,20180614,10234,17:09:27,0,0,310001,310001,37,4,20180614,Лог Сурки (по требованию),1-Обратное(37)-вечер
4,310001,310001,310001,20180614,20180614,10070,17:18:53,1,1,310001,310001,37,5,20180614,Хладокомбинат,1-Обратное(37)-вечер
5,310001,310001,310001,20180614,20180614,10071,17:22:06,10,3,310001,310001,37,6,20180614,Липецкое станкостроительное предприятие,1-Обратное(37)-вечер
6,310001,310001,310001,20180614,20180614,10072,17:27:48,1,4,310001,310001,37,7,20180614,Кольцо трубного завода (ул. Гагарина),1-Обратное(37)-вечер
7,310001,310001,310001,20180614,20180614,10073,17:29:30,0,0,310001,310001,37,8,20180614,Дом торговли,1-Обратное(37)-вечер
8,310001,310001,310001,20180614,20180614,10074,17:30:24,0,0,310001,310001,37,9,20180614,Титова,1-Обратное(37)-вечер
9,310001,310001,310001,20180614,20180614,10172,17:32:03,0,7,310001,310001,37,10,20180614,Ж/д вокзал (ул. Терешковой),1-Обратное(37)-вечер


In [23]:
# создаем поле ts, в котором указан timestamp прибытия тс на о.п.
df_asmpp['ts'] = list(map(lambda date, time: pd.datetime(date//10000, # year
                                                         date//100%100, # month 
                                                         date%100, # day
                                                         int(time.split(':')[0]), # hour
                                                         int(time.split(':')[1])), # minutes
                          df_asmpp.fact_trip_date,
                          df_asmpp.arrival_time))
# создаем поле date, в котором указана дата прибытия тс на о.п.
df_asmpp['date'] = df_asmpp.ts.dt.date

In [24]:
# корректируем название о.п.
# не исключена ситуация, что одному stop_id соответсвует несколько значений stop_name
# выбираем для каждого stop_id одно (последнее по дате) значения stop_name
if df_asmpp.stop_id.count() > 0:
    df_asmpp.stop_id.fillna(value=0, inplace=True)
    df_stops = df_asmpp.groupby(['stop_id','stop_name']).max()[['ts']].reset_index()
    df_stops = df_stops.groupby(['stop_id']).max()[['ts']].reset_index()\
        .join(df_stops.set_index(['stop_id','ts']), on=['stop_id','ts'])\
        .drop('ts', axis=1)
    # меняем названия о.п. в исходных данных
    df_asmpp = df_asmpp.drop('stop_name', axis=1)\
        .join(df_stops.set_index('stop_id'), on='stop_id')
else:
    #print('Значение поля stop_id не заполнено')
    #df_asmpp.stop_id.fillna(value=0, inplace=True)
    #df_asmpp['stop_id'] = df_asmpp.stop_name
    df_asmpp['stop_id'] = ''

In [25]:
# в первую очередь необходимо определить принадлежность данных к одному рейсу
# в настоящее время рейсы, начавшие движение в конце суток, а закончившие в начале следующих суток, разбиваются на два рейса
# для второй половины таких рейсов требуется корректировка значений поля date

# как оказалось grafic также может быть запонен с ошибками. при обработке данных за 201709 40 мвн имеют более 2 вариантов трассы
# можно попробовать дополнительно к grafic брать последние три цифры из vehicle_id
df_asmpp['veh_id'] = df_asmpp.vehicle_id.astype(str).str[-3:]
# как оказалось время прибытия тс на о.п. может быть указано с погрешностью 2-3 минуты, т.е. не исключена ситуация, 
# когда на следующий о.п. тс приезжает после того как была на предыдущем

# пока размышляю что с этим делать, есть ощущение, что необходимо ориентироваться на порядок строк в апи
# возможно следует вренуться к этой проблеме после обработки все данных из апи

# сортируем данные по времени прибытия тс на о.п.
df_asmpp.sort_values(['grafic', 'veh_id', 'ts','trip_num','stop_sequence'], inplace=True)
#df_asmpp.sort_values(['date','grafic','trip_num','stop_sequence'], inplace=True)
# убираем дубли
df_asmpp.drop_duplicates(subset=['grafic','veh_id','ts','stop_sequence'], inplace=True)
# определяем порядковый номер предыдущего о.п. для тс
df_asmpp['stop_seq_prev'] = df_asmpp.groupby('grafic').shift(1).stop_sequence
# если порядковый номер предыдущего о.п. больше либо равен чем порядковый номер текущего о.п., то начинаем новый рейс
df_asmpp['race_id'] = 0
df_asmpp.loc[~(df_asmpp.stop_seq_prev < df_asmpp.stop_sequence), 'race_id'] = 1
df_asmpp['race_id'] = df_asmpp.race_id.cumsum()
# убираем техническое поле
df_asmpp.drop('stop_seq_prev', axis=1, inplace=True)
# корректируем значение поля fact_trip_date
df_asmpp = df_asmpp.drop('date', axis=1).join(df_asmpp.groupby('race_id').min()['date'], on='race_id')

In [26]:
# находим всевозможные варианты поостановочной трассы
df_asmpp['trace'] = df_asmpp.stop_sequence.astype(str) + '-' + df_asmpp.stop_id.astype(str) + ',\n'
df_asmpp = df_asmpp.drop('trace', axis=1)\
                   .join(df_asmpp.groupby('race_id').apply(lambda x: x.astype(str).sum())[['trace']], on='race_id')
df_asmpp['trace'] = df_asmpp.trace.str[:-2]
#print('Число различных вариантов поостановочной трассы -- %d.' % len(df_asmpp['trace'].unique()))

In [35]:
df_asmpp['race_id']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
Name: race_id, dtype: int64

In [28]:
# формируем таблицу рейсов в которой содержится информация о дате и времени прибытия каждого рейса на первый о.п.
df_races = df_asmpp.groupby('race_id').min()[['trip_id','trace','date','ts']].reset_index()
# добавляем число пассажиров
df_races = df_races.join(df_asmpp.groupby('race_id').sum()[['pass_in','pass_out']], on=['race_id'])
df_races['pass'] = (df_races.pass_in + df_races.pass_out)/2
df_races.drop(['pass_in','pass_out'], axis=1, inplace=True)

# для каждого рейса определяем тип временного периода

# определяем принадлежность рейса к рабочему или выходному дню
df_races = df_races.join(df_workdays.set_index('date')['workday'], on='date')

#--------------
# определяем принадлежность рейса к сезону
df_races['season'] = 'неоп'
# формируем даты начала и окончания летнего сезона для каждого рейса
df_races['start'] = list(map(lambda t: pd.datetime(t.year, 
                                                   int(args['summer'][:2]),
                                                   int(args['summer'][3:5])), df_races.ts))
df_races['end']   = list(map(lambda t: pd.datetime(t.year, 
                                                   int(args['summer'][6:8]),
                                                   int(args['summer'][9:11])), df_races.ts))
# определяем принадлежность рейса к летнему сезону
df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'лето'
# формируем первый вариант даты начала и окончания зимнего сезона для каждого рейса
df_races['start'] = list(map(lambda t: pd.datetime(t.year, 
                                                   int(args['winter'][:2]),
                                                   int(args['winter'][3:5])), df_races.ts))
df_races['end']   = list(map(lambda t: pd.datetime(t.year + 1, 
                                                   int(args['winter'][6:8]),
                                                   int(args['winter'][9:11])), df_races.ts))
# определяем принадлежность рейса к зимнему сезону
df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'зима'
# формируем второй вариант даты начала и окончания зимнего сезона для каждого рейса
df_races['start'] = list(map(lambda t: pd.datetime(t.year - 1, 
                                                   int(args['winter'][:2]),
                                                   int(args['winter'][3:5])), df_races.ts))
df_races['end']   = list(map(lambda t: pd.datetime(t.year, 
                                                   int(args['winter'][6:8]),
                                                   int(args['winter'][9:11])), df_races.ts))
# определяем принадлежность рейса к зимнему сезону
df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'season'] = 'зима'
# убираем технические поля
df_races.drop(['start','end'], axis=1, inplace=True)
#---------------

# определяем принадлежность рейса к часу суток
df_races['time'] = ''
for i in range(len(args['time'])):
    start = args['time'][i-1]
    end   = args['time'][i]
    end = (pd.datetime(2000,1,1,int(end[:2]),int(end[-2:])) - pd.Timedelta(value=1, unit='m')).strftime('%H:%M')
    time = '%s-%s' % (start, end)
    if start < end:
        df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:])), df_races.ts))
        df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:])), df_races.ts))
        df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time
    else:
        df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:])), df_races.ts))
        df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:]))\
                                             + pd.Timedelta(value=1, unit='d'), df_races.ts))
        df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time
        df_races['start'] = list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(start[:2]),int(start[-2:]))\
                                            - pd.Timedelta(value=1, unit='d'), df_races.ts))
        df_races['end'] =   list(map(lambda t: pd.datetime(t.year,t.month,t.day,int(end[:2]),int(end[-2:])), df_races.ts))
        df_races.loc[(df_races.ts <= df_races.end)&(df_races.ts >= df_races.start), 'time'] = time

# задаем тип временного интервала
df_races['period_type'] = df_races.season.astype(str) + '-' +  df_races.workday.astype(str) + '-' + df_races.time.astype(str)

In [29]:
# формируем таблицу вариантов поостановочных трасс

# определяем даты, за которые есть данные асмпп по каждому из вариантов трассы и число рейсов
df_traces =\
    df_races.groupby(['trace']).min()[['date']]\
    .join(df_races.groupby(['trace']).max()[['date']], lsuffix='_first', rsuffix='_last')\
    .join(df_races.groupby(['trace']).count()[['race_id']])\
    .rename(columns={'race_id':'n_races'})\
    .sort_values(['date_last','n_races'], ascending=False)\
    .reset_index()
# создаем идентификатор варианта трассы
df_traces['trace_id'] = range(1, len(df_traces) + 1)
# добавляем идентификатор трассы в таблицу рейсов
df_races = df_races.join(df_traces.set_index('trace')['trace_id'], on='trace').drop('trace', axis=1)
# добавляем информацию о том как соотносятся между собой trace_id и trip_id 
df_tmp = df_races[['trip_id','trace_id']].drop_duplicates().set_index('trace_id').sort_values('trip_id')
df_tmp['trip_id'] = df_tmp.trip_id.astype(str) + ','
df_traces = df_traces.join(df_tmp.groupby(level='trace_id').apply(lambda x: x.astype(str).sum()), on='trace_id')
df_traces['trip_id'] = df_traces.trip_id.str[:-1]
# создаем поле, в котором будет отмечено следует ли использовать указанный вариант поостановочной трассы для построения матрицы
df_traces['flag'] = 0
# Определяем приоритетный вариант трассы по которому будет строиться матрица из следующих соображений:
# 1. Фиксируется варинат трассы, по которому имеются наиболее свежие данные асмпп (при наличии нескольких таких вариантов 
#    трассы, выбирается тот, у которого наибольшее число рейсов). Для зафиксированного варианта определяется дата первого
#    обследования.
# 2. Выделяются все варианты трассы, по которым дата последнего обследования более поздняя, чем зафикисрованная дата.
# 3. Из них выбирается вариант трассы с наибольшим числом обследований, если таких вариантов несколько,
#    то выбирается один вариант случайным образом.
date_first = df_traces.sort_values(['date_last','n_races'], ascending=False).iloc[0].date_first
mask = df_traces.date_last >= date_first
n_races = df_traces[mask].n_races.max()
df_traces.loc[(mask) & (df_traces.n_races == n_races), 'flag'] = 1

In [39]:
df_traces['trace']

0    1-10231,\n2-10232,\n3-10233,\n4-10234,\n5-1007...
Name: trace, dtype: object

In [30]:
# определяем число рейсов по каждому из вариантов трассы за каждый тип временного интервала
df_nraces = df_races.groupby(['trace_id','season','workday','time']).count()['race_id']\
    .unstack('trace_id')\
    .fillna(value=0)\
    .astype(int)\
    .reset_index()\
    .sort_values(['season','workday','time'], ascending=[True,False,True])

In [31]:
# определяем число пассажиров, перевезенных средним рейсом по каждому из вариантов трассы за каждый тип временного интервала
df_pass =(\
    df_races.groupby(['trace_id','season','workday','time']).mean()['pass']\
        .unstack('trace_id')\
        .fillna(value=0)\
        .round(1)\
        .astype(str)\
    + ' ± ' +\
    df_races.groupby(['trace_id','season','workday','time']).std()['pass']\
        .unstack('trace_id')\
        .fillna(value=0)\
        .round(1)\
        .astype(str)\
    ).reset_index()\
    .sort_values(['season','workday','time'], ascending=[True,False,True])

In [32]:
# вычисляем посадку и высадку среднего рейса по всем типам временных интервалов и вариантам поостановочной трассы
df_inout =\
    df_asmpp[['mvn','race_id','stop_sequence','stop_id','stop_name','pass_in','pass_out']]\
    .join(df_races.set_index('race_id')[['trace_id','period_type']], on='race_id')\
    .groupby(['mvn','trace_id','period_type','stop_sequence','stop_id','stop_name']).mean()[['pass_in','pass_out']]\
    .reset_index()

In [33]:
# формируем xlsx файл с результатами предобработки
xls_writer = pd.ExcelWriter(args['results'] + '/'+ mvn + '.xlsx')

# сохраняем вкладку inout
df_inout.to_excel(xls_writer, index=False, sheet_name='inout')
xls_sheet = xls_writer.sheets['inout']
xls_sheet.set_column('A:A', len(mvn) + 1, None)
xls_sheet.set_column('C:C', 21, None)
xls_sheet.set_column('F:F', (df_inout.stop_name.str.len()).max() + 1, None)
xls_fmt = xls_writer.book.add_format(properties={'num_format': '0.00'})
xls_sheet.set_column('G:H', None, xls_fmt)

# сохраняем вкладку traces
df_traces.set_index('trace_id')[['n_races','date_first','date_last','flag','trip_id','trace']]\
    .to_excel(xls_writer, sheet_name='traces')
xls_sheet = xls_writer.sheets['traces']
xls_sheet.set_column('C:D', 11, None)

# сохраняем вкладку nraces
df_nraces.to_excel(xls_writer, index=False, sheet_name='nraces')
xls_sheet = xls_writer.sheets['nraces']
xls_sheet.set_column('C:C', 11, None)
xls_sheet.set_column('D:ZZ', 4, None)

# сохраняем вкладку pass
df_pass.to_excel(xls_writer, index=False, sheet_name='pass')
xls_sheet = xls_writer.sheets['pass']
xls_sheet.set_column('C:C', 11, None)
xls_sheet.set_column('D:ZZ', 14, None)

# сохраняем и закрываем файл
xls_writer.save()

In [34]:
%return len(df_traces)

1